In [10]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['번역(영-한)', '번역(한-영)', '번역(한-중)', '번역(한-일)', '영어 번역 교정', '중국어 번역 교정', '일본어 번역 교정']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            source_language_key = 'source_language_code' if 'source_language_code' in line else 'SOURCE_LANGUAGE_CODE'
            target_language_key = 'target_language_code' if 'target_language_code' in line else 'TARGET_LANGUAGE_CODE'
            source_language = line[source_language_key]
            source_text = line['SOURCE_SENTENCE']
            target_language = line[target_language_key]
            
            target_text = line['TARGET_SENTENCE']
            if 'MT_SENTENCE' in line:
                target_text_mt = line['MT_SENTENCE']
                target_texts = [target_text, target_text_mt]
            #### data preprocess end 
            
            if source_language == 'en' and target_language == 'ko':
                task_id = 0
            elif source_language == 'ko' and target_language == 'en':
                task_id = 1
            elif source_language == 'ko' and target_language == 'zh':
                task_id = 2
            elif source_language == 'ko' and target_language == 'ja':
                task_id = 3
                
            #### 번역
            data = {'input': None, 'output': None}
            ## preprocess data from line
            data = {'input': {}, 'output': None}
            data['input']['source_text'] = source_text
            data['input']['source_language'] = source_language
            data['input']['target_language'] = target_language
            data['output'] = target_text
            ## preprocess data from line end
            task_files[task_id].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 번역 end
            
            if ('MT_SENTENCE' in line) and (target_text != target_text_mt):
                if target_language == 'en':
                    task_id = 4
                elif target_language == 'zh':
                    task_id = 5
                elif target_language == 'ja':
                    task_id = 6
                else:
                    continue
                
                data['output'] = target_texts
                task_files[task_id].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 28/28 [00:01<00:00, 16.29it/s]
